In [1]:
# third party
import torch as th

# syft absolute
import syft as sy
from syft.core.node.common.service.obj_search_permission_service import (
    ImmediateObjectSearchPermissionUpdateService,
)
from syft.core.node.common.service.obj_search_permission_service import (
    ObjectSearchPermissionUpdateMessage,
)

INITIALIZING IN MEMORY STORE!!!


In [2]:
bob_phone = sy.Device(name="Bob's iPhone")
bob_phone_client = bob_phone.get_client()

ptr = th.tensor([1, 2, 3]).send(bob_phone_client)

msg = ObjectSearchPermissionUpdateMessage(
    add_instead_of_remove=True,
    target_verify_key=bob_phone_client.verify_key,
    target_object_id=ptr.id_at_location,
    address=bob_phone_client.address,
)

ImmediateObjectSearchPermissionUpdateService.process(
    node=bob_phone, msg=msg, verify_key=bob_phone.verify_key
)

INITIALIZING IN MEMORY STORE!!!
SaveObjectAction <Storable: tensor([1, 2, 3])>
Storing object:<UID: 19e7f83675ed48409cd7c82077749ed3> -> <Storable: tensor([1, 2, 3])>
Storing object:<UID: 19e7f83675ed48409cd7c82077749ed3> -> <Storable: tensor([1, 2, 3])>
Deleting object:<UID: 19e7f83675ed48409cd7c82077749ed3>


In [3]:
bob_phone.store[ptr.id_at_location].search_permissions

{<syft.core.common.group._create_VERIFYALL.<locals>.VerifyAll at 0x7fbab8794400>: <syft.lib.python._SyNone at 0x7fbaf8de7c40>,
 <nacl.signing.VerifyKey at 0x7fbab8cb4e50>: <UID: 6032dd1262ff4989b773b37c79889a84>}

In [4]:
bob_phone.store[ptr.id_at_location].search_permissions[bob_phone_client.verify_key]


<UID: 6032dd1262ff4989b773b37c79889a84>

In [5]:
msg.id

<UID: 6032dd1262ff4989b773b37c79889a84>